In [1]:
# NOTE RATING IS 0-5 INTEGER ONLY

In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from itertools import islice
import csv

In [2]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [5]:
def readCSV(path):
    with open(path, mode='r') as file:
        reader = csv.reader(file)
        # Skip the header row
        next(reader, None)
        # Loop through the rows
        for row in reader:
            # Extract the first 4 columns
            u,i,d,r = row[:4]
            yield u, i, d, float(r)


### This is a temporary version of the alternative model. 
Feel free to play with the data! To avoid merge conflicts though either duplicate the file or use a different branch.
If there are any issues or fixes to stuff that I've done lmk.

The baseline is just here to chill around. 


In [6]:
train_dataset = []
ratings_dict = defaultdict(int)
itemsPerUser = defaultdict(set)
usersPerItem = defaultdict(set)
global_fallback_rating = 0
for l in readCSV("data/interactions_train.csv"):
    train_dataset.append(l)
    user, recipe, data, rating = l
    itemsPerUser[user].add(recipe)
    usersPerItem[recipe].add(user)
    global_fallback_rating += rating

In [7]:
global_fallback_rating = global_fallback_rating / len(train_dataset)
global_fallback_rating
def baseline_predictor(user, item, date):
    return global_fallback_rating

In [8]:
validation_dataset = []
validation_reals = []
validation_preds = []
for l in readCSV("data/interactions_validation.csv"):
    validation_dataset.append(l)
    user, recipe, date, rating = l
    validation_reals.append(rating)
    validation_preds.append(baseline_predictor(user, recipe, date))
print(f'baseline MSE: {mean_squared_error(validation_reals, validation_preds)}')

baseline MSE: 1.8138061805801156


### Alternative Model

In [9]:
#This is a model using svdpp to compare our original model to
import gzip
from collections import defaultdict
import random
import csv
import numpy as np
import math
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise import SVDpp
from surprise.model_selection import train_test_split
from surprise import accuracy

#using pandas to make display a bit easier to debug
df = pd.read_csv("data/interactions_train.csv", names=['user_id', 'recipe', 'date', 'rating', "u", "i"], skiprows = 1 )
print(df[0:10])
df['rating'] = df['rating'].astype(int)  
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['user_id', 'recipe','rating']], reader)
trainset, testset = train_test_split(data, test_size=0.5)
model = SVDpp(n_factors=3, lr_all=0.01, reg_all = 0.2)
model.fit(trainset)

#validation
pairs = pd.read_csv('data/interactions_validation.csv', names=['user_id', 'recipe', 'date', 'rating', "u", "i"], skiprows=1)
predictions = []
for _, row in pairs.iterrows():
    user = row['user_id']
    rec = row['recipe']
    pred = model.predict(user, rec).est
    predictions.append(pred)

pairs['prediction'] = predictions
predictions_test = model.test(testset)
mse = accuracy.mse(predictions_test)
print(f'MSE on test set: {mse}')

validation_actual = pairs['rating'].astype(float)  
validation_predicted = pairs['prediction']        
validation_mse = np.mean((validation_actual - validation_predicted) ** 2)
print(f'MSE on validation set: {validation_mse}')

   user_id  recipe        date  rating      u       i
0     2046    4684  2000-02-25     5.0  22095   44367
1     2046     517  2000-02-25     5.0  22095   87844
2     1773    7435  2000-03-13     5.0  24732  138181
3     1773     278  2000-03-13     4.0  24732   93054
4     2046    3431  2000-04-07     5.0  22095  101723
5     2046   13307  2000-05-21     5.0  22095  134551
6     2312     780  2000-09-12     5.0   1674  127175
7     2312   51964  2000-09-26     5.0   1674  151793
8     2312    1232  2000-10-17     4.0   1674   15498
9     2312    4397  2000-10-17     5.0   1674   14380
MSE: 0.8513
MSE on test set: 0.8512937027458124
MSE on validation set: 1.6628329128833175


In [ ]:
#This is a scrapped list of parameter grid I have run so far, I will be replacing these with lists generated by np later

'''param_grid = {
    'n_factors': [1],
    'lr_all': [0.002 , 0.005, 0.01, 0.015, 0.2],
    'reg_all': [0.02, 0.05, 0.1, 0.2, 0.25, 0.3]
}
'''
'''param_grid = {
    'n_factors': [1],
    'lr_all': [0.01, 0.00975, 0.0095],
    'reg_all': [0.25]
}
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=3)
gs.fit(data)
print("Best NF", gs.best_params['rmse'])
results = gs.cv_results
for params, mean_rmse, std_rmse in zip(results['params'], results['mean_test_rmse'], results['std_test_rmse']):
    print(f"Params: {params}, Mean RMSE: {mean_rmse:.4f}, Std Dev RMSE: {std_rmse:.4f}")'''